# HW 5
Shadrunov Aleksey, BIB201

# задача:  
кластеризовать песни разных исполнителей по текстам. проверить, получается ли отгадывать правильный кластер песни по тексту.

для решения используем **TF-IDF** метод вместе с **KMeans**.  
для получения текстов используем парсинг google поиска (**beautifulsoup**).

сначала попробуем получить результат на маленьких тестовых датасетах, затем на основном.

# parsers, classifiers, cleaners

определим нужные функции для скачивания песен, очистки датасета и классификации

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import numpy as np
from scipy import stats

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

In [31]:
# parse google search results
def extract_text(soup: BeautifulSoup) -> str:
    """extracts text from beautifulsoup object

    :param soup: beautiful soup object (div main)
    :type soup: BeautifulSoup
    :return: text of the song
    :rtype: str
    """
    main = soup.find("div", id="main")

    # select elements containing text of the song
    elements = main.findChildren(lambda e: "{" not in e.text and "\n" in e.text and "/" not in e.text)
    lengths = [len(i.text) for i in elements]
    unique_lengths = list(set(lengths))

    # select element of the second size
    unique_lengths.sort()
    text_len = unique_lengths[-2]

    # get text from selected element
    text = main.findChild(
        lambda e: len(e.text) == text_len and "{" not in e.text and "\n" in e.text and "/" not in e.text
    ).text
    return text


url = "https://www.google.com/search"

# create dataframe with lyrics from google
def create_dataframe_with_songs(filename_r: str = "songs.csv", filename_w: str = "songs_texts.csv") -> pd.DataFrame:
    """reads csv with artists and song titles, dowloads lyrics and stores the result into another file

    :param filename_r: name of given csv, defaults to "songs.csv"
    :type filename_r: str, optional
    :param filename_w: name of csv to store data, defaults to "songs_texts.csv"
    :type filename_w: str, optional
    :return: dataframe with texts
    :rtype: pd.DataFrame
    """
    songs = pd.read_csv(filename_r)

    texts = []

    for i in range(len(songs)):
        print(songs["title"][i])        
        if not pd.isna(songs["text"][i]):
            text = songs["text"][i]
        else:
            params = {"q": songs["artist"][i] + " " + songs["title"][i] + " lyrics"}
            page = requests.get(url, params)
            soup = BeautifulSoup(page.content, "html.parser")
            try:
                text = extract_text(soup)
            except:
                text = np.nan
                print("Error with parsing")

        texts.append(text)
    
    print(f"tried to download {len(texts)} lyrics")
    songs["text"] = texts
    songs = songs.dropna()
    print(f"dropped {len(texts) - len(songs)} rows from dataframe")

    songs.to_csv(filename_w)
    return songs


# clean dataset
def clean_dataset(dataset: pd.DataFrame) -> pd.DataFrame:
    """lowercase, only letters and digits, remove empty elements

    :param dataset: _description_
    :type dataset: pd.DataFrame
    :return: _description_
    :rtype: pd.DataFrame
    """
    clear_text = []

    for text in dataset["text"]:
        # lower case
        text = text.lower().split()
        # remove non-alphabetical symbols
        text = ["".join(c for c in s if c.isalnum()) for s in text]
        # remove empty strings
        text = list(filter(lambda x: x, text))

        clear_text.append(" ".join(text))

    dataset["clear_text"] = clear_text

    return dataset[["artist", "clear_text"]]


In [3]:
# ML
def clusterize(dataset, n_clusters=2):
    """проводит кластеризацию. по входному корпусу текстов строит TF-IDF статистику,
    а затем применяет ее к KMeans-алгоритму. после обучения можно использовать 
    созданные объекты для дальнейших предсказаний.

    :param dataset: входной корпус текстов
    :type dataset: df.Series
    :param n_clusters: число кластеров в KMeans, defaults to 2
    :type n_clusters: int, optional
    :return: результат предсказания и объекты векторизатора и kmeans
    :rtype: tuple
    """
    vectorizer = TfidfVectorizer(stop_words="english")
    X = vectorizer.fit_transform(dataset)
    kmeans = KMeans(n_clusters=n_clusters, max_iter=100000, tol=0.0001, init="k-means++")
    return kmeans.fit_predict(X), vectorizer, kmeans


# тестовый датасет с латинским и британским исполнителем

для начала возьмем датасет с песнями английской группы *One Direction* и испанским исполнителем *Miki Núñez*. очевидно, что у песен этих исполнителей отличаются языки, что должно обеспечить идеальную кластеризацию и точность предсказаний. проверим:

In [4]:
# долгая операция, выкачиваются тексты
spanish = create_dataframe_with_songs("spanish/songs.csv", "spanish/songs_texts.csv")

celebrate
eterno verano
nadie se salva
per tu
coral del arrecife
apaga la luz
la venda
tanto tiempo
la ultima palabra
la cabana
what makes you beautiful
best song ever
steal my girl
drag me down
story of my life
kiss you
little things
perfect
history
Error with parsing
one thing
they don't know about us
one way or another
midnight memories
night changes
no control
tried to download 25 lyrics
dropped 1 rows from dataframe


In [5]:
spanish.head()  # результат

,artist,title,text
0,miki nunez,celebrate,"Hoy, ya sķ, que debo regalarme\nTodo aquello q..."
1,miki nunez,eterno verano,"EmpezarŪa con con un ""me levantů""\nPero no ten..."
2,miki nunez,nadie se salva,"Nadie se salva\n\nNadie se salva\n\nMira, yo n..."
3,miki nunez,per tu,Quan s'enfonsa tot\nSurten flors de las esquer...
4,miki nunez,coral del arrecife,En esta isla de arena blanca y fina\nHe visto ...


In [6]:
spanish = clean_dataset(spanish)

In [7]:
display(spanish.head())
display(spanish.tail())

,artist,clear_text
0,miki nunez,hoy ya sķ que debo regalarme todo aquello que ...
1,miki nunez,empezarūa con con un me levantů pero no tengo ...
2,miki nunez,nadie se salva nadie se salva mira yo no quier...
3,miki nunez,quan senfonsa tot surten flors de las esquerda...
4,miki nunez,en esta isla de arena blanca y fina he visto a...


,artist,clear_text
20,one direction,people say we shouldnt be together were too yo...
21,one direction,one way or another im gonna find ya im gonna g...
22,one direction,one two three straight off the plane to a new ...
23,one direction,goin out tonight changes into something red he...
24,one direction,oh oh oh ooh ooh stained coffee cup just a fin...


In [8]:
res, vectorizer, kmeans = clusterize(spanish["clear_text"])
res

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int32)

видим, что в результате кластеризации и последующем применении предсказания первые 10 измерений попали в кластер 1, а следующие 15 — в кластер 0. таким образом, в рассмотренном тривиальном случае алгоритм отрабатывает идеально, классифицируя песни без ошибок.

# тестовый датасет с репом и британским исполнителем

теперь рассмотрим датасет с разными по жанру композициями: песнями английской группы *One Direction* и репером *Lil Wayne*. репу свойственна особая лексика, что мы и проверим далее:

In [9]:
rap = create_dataframe_with_songs("rap/songs.csv", "rap/songs_texts.csv")
display(rap.head())
display(rap.tail())

3 peat
mr carter
a milli
got money
phone home
shoot me down
tie my hand
lollipop
let the beat build
mrs officer
what makes you beautiful
best song ever
steal my girl
drag me down
story of my life
kiss you
little things
perfect
history
Error with parsing
one thing
they don't know about us
one way or another
midnight memories
night changes
no control
tried to download 25 lyrics
dropped 1 rows from dataframe


,artist,title,text
0,lil wayne,3 peat,"Yessir! They can't stop me, even if they stopp..."
1,lil wayne,mr carter,"Yo\nYo, Drew and Inf', did this, this right he..."
2,lil wayne,a milli,Young Money! You dig? Mack I'm going in\n\nA m...
3,lil wayne,got money,"Yeah, yeah!\nI need a Winn-Dixie grocery bag f..."
4,lil wayne,phone home,"This is, this is, this is, this is\nWe are not..."


,artist,title,text
20,one direction,they don't know about us,People say we shouldn't be together\nWe're too...
21,one direction,one way or another,"One way or another, I'm gonna find ya\nI'm gon..."
22,one direction,midnight memories,"One, two, three\n\nStraight off the plane to a..."
23,one direction,night changes,"Goin' out tonight, changes into something red\..."
24,one direction,no control,"Oh, oh, oh\n(Ooh, ooh)\n\nStained coffee cup\n..."


In [10]:
# очистим датасет от заглавных букв и прочих символов
rap = clean_dataset(rap)
display(rap.head())
display(rap.tail())

,artist,clear_text
0,lil wayne,yessir they cant stop me even if they stopped ...
1,lil wayne,yo yo drew and inf did this this right here is...
2,lil wayne,young money you dig mack im going in a million...
3,lil wayne,yeah yeah i need a winndixie grocery bag full ...
4,lil wayne,this is this is this is this is we are not the...


,artist,clear_text
20,one direction,people say we shouldnt be together were too yo...
21,one direction,one way or another im gonna find ya im gonna g...
22,one direction,one two three straight off the plane to a new ...
23,one direction,goin out tonight changes into something red he...
24,one direction,oh oh oh ooh ooh stained coffee cup just a fin...


In [11]:
# кластеризуем
res, vectorizer, kmeans = clusterize(rap["clear_text"])
res

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1], dtype=int32)

в исходном датасете первые 10 песен — реп, остальные 15 — британская группа. в результате кластеризации и проверки отнесения песен к кластерам видим, что из первых 10 песен 8 отнесены к классу 0. в остальных песнях допущена 1 ошибка.  

это подтверждает гипотезу, что репу свойственна характерная лексика, отличающаяся от поп-музыки и позволяющая провести кластеризацию.

можем рассчитать метрики:  
(**Precision** — how many selected items are relevant?  
**Recall** — how many relevant items are selected?)

In [12]:
print(
    "Accuracy:",
    accuracy_score(
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
    ),
)

print("Precision of rap:", (10 - 2) / 10)
print("Recall of rap:", (10 - 2) / 10)

Accuracy: 0.875
Precision of rap: 0.8
Recall of rap: 0.8


# большой датасет с 11 поп-исполнителями

теперь рассмотрим датасет с 11 поп-исполнителями и реп исполнителем из предыдущего пункта. предположительно, все эти песни будут схожими в отношении лексики и успешной кластеризации не выйдет, однако, попробуем

In [13]:
big = create_dataframe_with_songs("big/songs.csv", "big/songs_texts.csv")
display(big.head())
display(big.tail())

love story taylor's version
anti-hero
lover
blank space
bejeweled
you need to calm down
all too wait taylor's version
shake it off
style
i knew you were trouble taylor's version
me
wildes dreams
lavender haze
karma
willow
one of the boys
i kissed a girl
thinking of you
mannequin
ur so gay
hot n coldd
if you can afford me
lost
self inflicted
i'm still breathing
fingerprints
smile
cry about it later
daisies
never really over
waterloo
mamma mia
dancing queen
i still have faith in you
when you danced with me
little things
take a chance on me
just a notion
i can be that woman
the winner take it all
super trouper
don't shut me down
voulez-vous
money money money
slipping through my fingers
as it was
late night talking
adore you
watermelon sugar
misic for a sushi restaurant
daylight
matilda
golden
sign of the times
satellite
love of my life
lights up
falling
kiwi
fine line
bigger than me
out of my system
silver tongues
written all over your face
back to you
always you
Error with parsing
miss y

,artist,title,text
0,taylor swift,love story taylor's version,We were both young when I first saw you\nI clo...
1,taylor swift,anti-hero,I have this thing where I get older but just n...
2,taylor swift,lover,We could leave the Christmas lights up 'til Ja...
3,taylor swift,blank space,"Nice to meet you, where you been?\nI could sho..."
4,taylor swift,bejeweled,"Baby love, I think I've been a little too kind..."


,artist,title,text
165,lil wayne,shoot me down,Open up your hearts people\nPage one chapter o...
166,lil wayne,tie my hand,We are at war\nWith the universe\nThe sky is f...
167,lil wayne,lollipop,"Ow, uh-huh, no homo (Young Mula, baby)\nI said..."
168,lil wayne,let the beat build,Yeah\nI see you big bro\nI'ma kill these nigga...
169,lil wayne,mrs officer,"Hey...\nHey... (yeah)\nHey... (yeahh)\nHey, he..."


In [14]:
# очистим датасет от заглавных букв и прочих символов
big = clean_dataset(big)
display(big.head())
display(big.tail())

,artist,clear_text
0,taylor swift,we were both young when i first saw you i clos...
1,taylor swift,i have this thing where i get older but just n...
2,taylor swift,we could leave the christmas lights up til jan...
3,taylor swift,nice to meet you where you been i could show y...
4,taylor swift,baby love i think ive been a little too kind d...


,artist,clear_text
165,lil wayne,open up your hearts people page one chapter on...
166,lil wayne,we are at war with the universe the sky is fal...
167,lil wayne,ow uhhuh no homo young mula baby i said hes so...
168,lil wayne,yeah i see you big bro ima kill these niggas m...
169,lil wayne,hey hey yeah hey yeahh hey hey hey valentine w...


In [15]:
artists = set(big["artist"])
artists_num = len(artists)
artists_num, artists

(12,
 {'abba',
  'adele',
  'arctic monkeys',
  'ashe',
  'billie eilish',
  'harry styles',
  'katy perry',
  'lil wayne',
  'louis tomlinson',
  'miley cyrus',
  'one direction',
  'taylor swift'})

In [29]:
# кластеризуем
res, vectorizer, kmeans = clusterize(big["clear_text"], n_clusters=artists_num)
res

array([ 3, 10,  5,  3,  5,  7,  8,  2, 10,  7,  2,  9,  0,  6,  5,  3,  3,
        1,  5,  3,  5,  3,  6,  2,  1,  8,  8,  2, 10,  6,  2,  3, 10,  3,
        2,  9,  3,  5,  5,  3,  2,  1,  3, 10,  1,  3,  1,  9,  3,  3,  6,
        3,  3,  1,  3,  3,  3,  9,  7,  0,  3,  3,  8,  7,  3,  1,  5, 10,
        1,  5,  3,  7,  2,  3,  3,  3,  7,  9,  7, 11,  9,  3,  1,  3,  1,
        1,  5,  8,  1,  0,  3, 11,  5,  6,  0,  1,  3,  3,  3,  4,  5,  7,
        0,  9,  2,  3,  3,  2,  3,  2,  7,  3,  8,  8,  0,  3,  2,  4,  0,
        1, 11,  8,  3,  1,  3,  3,  3,  6,  7,  5,  6,  8,  3,  8,  3,  6,
        2,  9,  2,  6,  3,  0,  9,  8,  1,  1,  5,  3,  8,  6,  0,  8,  5,
       10, 10, 10, 10,  8, 10,  3,  0, 10,  6], dtype=int32)

видим, что кластеризация не такая однородная, как в предыдущем случае. попробуем посчитать правильность для каждого класса.

In [30]:
print("{:<18} {:>3}   {:>5}   {:<}".format("artist", "label", "freq", "clusters"))
for a in artists:
    artist_songs = list(big[big["artist"] == a]["clear_text"])
    res = kmeans.predict(vectorizer.transform(artist_songs))
    mode = stats.mode(res, keepdims=False)
    mode_value = mode[0]
    freq = mode[1] / len(res)
    # print(a, res, mode[0], mode[1] / len(res))
    print("{:<18} {:>3}     {:>5.3f}  {:<}".format(a, mode_value, freq, str(res)))


artist             label    freq   clusters
adele                8     0.200  [2 6 3 0 9 8 1 1 5 3 8 6 0 8 5]
louis tomlinson      3     0.333  [ 3  3  8  7  3  1  5 10  1  5  3  7]
one direction        3     0.357  [3 3 4 5 7 0 9 2 3 3 2 3 2 7]
miley cyrus          3     0.231  [ 3  8  8  0  3  2  4  0  1 11  8  3  1]
lil wayne           10     0.600  [10 10 10 10  8 10  3  0 10  6]
billie eilish        1     0.333  [ 1  3  1  1  5  8  1  0  3 11  5  6  0  1  3]
arctic monkeys       3     0.357  [3 3 3 6 7 5 6 8 3 8 3 6 2 9]
abba                 3     0.333  [ 2  3 10  3  2  9  3  5  5  3  2  1  3 10  1]
taylor swift         5     0.200  [ 3 10  5  3  5  7  8  2 10  7  2  9  0  6  5]
harry styles         3     0.533  [3 1 9 3 3 6 3 3 1 3 3 3 9 7 0]
katy perry           3     0.267  [ 3  3  1  5  3  5  3  6  2  1  8  8  2 10  6]
ashe                 3     0.400  [ 2  3  3  3  7  9  7 11  9  3]


снова видим, что Lil Wayne классифицирован наиболее точно (0.6)

# выводы

проанализировав полученные результаты, можем заметить, что кластеризация на тривиальных примерах работает хорошо, однако с увеличением количества кластеров и их взаимной схожести результат работы алгоритма ухудшается.  

наиболее просто классифицируется исполнитель **репа**, что можно объяснить характерной лексикой, присущей этому жанру. поп-исполнители в целом классифицированы плохо, наибольший результат у Гарри Стайлса — частота встречаемости самого частого кластера в его песнях составляет 0.533. возможно, это связано с тем, что артист использует необычные слова в своих текстах. однако, хотя больше половины песен Гарри Стайлса и попали в кластер 3, в этот же кластер попали очень многие песни других поп-исполнителей. Возможно, Гарри — наиболее "характерный" представитель среди всех рассмотренных артистов.